# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [31]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [32]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [33]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [6]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [7]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [8]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [9]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [10]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [11]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [12]:
distances = pdist(prod_cust_pivot, "euclidean")
Q = squareform(distances)
similarity_matrix = pd.DataFrame(1/(1+Q), index=prod_cust_pivot.index, columns=prod_cust_pivot.index)
similarity_matrix

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.881127,0.882219,0.884604,0.890449,0.879859,0.885648,0.880852,0.886501,0.890573,...,0.884117,0.888124,0.886567,0.879578,0.875696,0.878427,0.886163,0.886564,0.887956,0.878223
"Appetizer - Mini Egg Roll, Shrimp",0.881127,1.000000,0.883877,0.880604,0.887930,0.881577,0.885241,0.890196,0.888646,0.883484,...,0.887569,0.887831,0.888033,0.881019,0.878720,0.871915,0.888702,0.890837,0.887661,0.882616
Appetizer - Mushroom Tart,0.882219,0.883877,1.000000,0.884708,0.883266,0.884124,0.886495,0.886072,0.887707,0.881019,...,0.886796,0.885968,0.885056,0.887445,0.882789,0.879286,0.890062,0.889960,0.889397,0.881077
Appetizer - Sausage Rolls,0.884604,0.880604,0.884708,1.000000,0.885025,0.885020,0.881992,0.883090,0.880745,0.881226,...,0.882793,0.886507,0.884984,0.886515,0.877195,0.868976,0.885470,0.884688,0.883661,0.877701
Apricots - Dried,0.890449,0.887930,0.883266,0.885025,1.000000,0.886792,0.887542,0.889058,0.886234,0.887162,...,0.886223,0.889001,0.890087,0.887478,0.878571,0.872542,0.892076,0.891209,0.889794,0.886477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.878427,0.871915,0.879286,0.868976,0.872542,0.875465,0.876566,0.877387,0.877911,0.876792,...,0.882272,0.877286,0.874760,0.872730,0.869942,1.000000,0.876589,0.875700,0.877148,0.871735
Yoghurt Tubes,0.886163,0.888702,0.890062,0.885470,0.892076,0.887069,0.893496,0.889616,0.888383,0.888293,...,0.891366,0.892520,0.890222,0.884474,0.885830,0.876589,1.000000,0.889996,0.891179,0.885087
"Yogurt - Blueberry, 175 Gr",0.886564,0.890837,0.889960,0.884688,0.891209,0.891202,0.890707,0.889430,0.891434,0.886262,...,0.890363,0.892818,0.890805,0.885548,0.880785,0.875700,0.889996,1.000000,0.890230,0.886320


### Step 2: Get the products purchased for a specific customer of your choice.

In [13]:
cust = cust_prod_pivot.T
prodcuts = list(cust.loc[cust[639] != 0].index)

### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [14]:
product_list=[]
for p in prodcuts:
    similar_products = list(similarity_matrix.sort_values(by=[p], ascending=False).head(6).index)
    similar_products = similar_products[1:-1]
    product_list.append(similar_products)
product_list = [p for x in product_list for p in x]


In [15]:
for p in product_list: 
    if p in prodcuts: 
        product_list.remove(p)
product_list

['Broom - Corn',
 'Beets - Mini Golden',
 'Oil - Shortening - All - Purpose',
 'Rosemary - Primerba, Paste',
 'Cheese - Brie,danish',
 'Meldea Green Tea Liquor',
 'Oil - Shortening - All - Purpose',
 'Spinach - Baby',
 'Puree - Passion Fruit',
 'Oil - Shortening - All - Purpose',
 'Sun - Dried Tomatoes',
 'Yogurt - Blueberry, 175 Gr',
 'Table Cloth - 53x69 Colour',
 'Bread Foccacia Whole',
 'Cookies - Assorted',
 'Wine - Crozes Hermitage E.',
 'Towels - Paper / Kraft',
 'Soup - Campbells, Beef Barley',
 'Cookies - Assorted',
 'Oil - Shortening - All - Purpose',
 'Duck - Breast',
 'Yogurt - Blueberry, 175 Gr',
 'Garlic - Peeled',
 'Pastry - Cheese Baked Scones',
 'Cod - Black Whole Fillet',
 'Sun - Dried Tomatoes',
 'French Pastry - Mini Chocolate',
 'Kiwi',
 'Rosemary - Primerba, Paste',
 'Veal - Osso Bucco',
 'Chocolate - Semi Sweet, Calets',
 'Cookies - Assorted',
 'Bread - Calabrese Baguette',
 'Yogurt - Blueberry, 175 Gr',
 'Spinach - Baby',
 'Cookies - Assorted',
 'Oil - Shortenin

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [16]:
from collections import Counter

popular_products = dict(Counter(product_list))
popular_products = sorted(popular_products.items(), key=lambda x: x[1], reverse=True)
popular_products = popular_products[0:4]
for i in range(len(popular_products)): 
    print(popular_products[i][0])

Oil - Shortening - All - Purpose
Cookies - Assorted
Rosemary - Primerba, Paste
Spinach - Baby


### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [17]:
r_products = {}
customer_ids=set(data["CustomerID"])
for c in customer_ids: 
    cust = cust_prod_pivot.T
    prodcuts = list(cust.loc[cust[c] != 0].index)
    product_list=[]
    for p in prodcuts:
        similar_products = list(similarity_matrix.sort_values(by=[p], ascending=False).head(6).index)
        similar_products = similar_products[1:-1]
        product_list.append(similar_products)
    product_list = [p for x in product_list for p in x]
    popular_products = dict(Counter(product_list))
    popular_products = sorted(popular_products.items(), key=lambda x: x[1], reverse=True)
    popular_products = popular_products[:5]
    l=[]
    for i in range(len(popular_products)): 
        l.append(popular_products[i][0])
    r_products[c]=l

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [18]:
recomendations_df=pd.DataFrame(r_products).T

recomendations_df

,0,1,2,3,4
83973,"Rosemary - Primerba, Paste",Rice - Jasmine Sented,Oil - Shortening - All - Purpose,"Chocolate - Semi Sweet, Calets",Muffin - Zero Transfat
59399,"Rosemary - Primerba, Paste",Oil - Shortening - All - Purpose,Sun - Dried Tomatoes,Spinach - Baby,"Yogurt - Blueberry, 175 Gr"
92168,Hersey Shakes,Cookies - Assorted,Sun - Dried Tomatoes,Flavouring - Orange,"Bar Mix - Pina Colada, 355 Ml"
49159,Oil - Shortening - All - Purpose,"Rosemary - Primerba, Paste",Spinach - Baby,Towels - Paper / Kraft,Sun - Dried Tomatoes
18441,"Rosemary - Primerba, Paste",V8 - Berry Blend,Oil - Shortening - All - Purpose,Cookies - Assorted,Spinach - Baby
...,...,...,...,...,...
77814,"Rosemary - Primerba, Paste",Oil - Shortening - All - Purpose,Sun - Dried Tomatoes,Towels - Paper / Kraft,Rice - Jasmine Sented
32753,"Rosemary - Primerba, Paste",Sun - Dried Tomatoes,Cookies - Assorted,Oil - Shortening - All - Purpose,V8 - Berry Blend
20476,"Hickory Smoke, Liquid","Rosemary - Primerba, Paste","Chocolate - Semi Sweet, Calets",Spinach - Baby,Oil - Shortening - All - Purpose
67582,Sun - Dried Tomatoes,Oil - Shortening - All - Purpose,"Yogurt - Blueberry, 175 Gr","Chocolate - Semi Sweet, Calets",Beef - Montreal Smoked Brisket


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [19]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [34]:
products = new_customer.keys()

In [35]:
cust_prod_pivot = cust_prod_pivot.append(new_customer, ignore_index=True)

In [38]:
cust_prod_pivot = cust_prod_pivot.fillna(0)

In [44]:
distances = pdist(cust_prod_pivot, "euclidean")
Q = squareform(distances)
similarity_matrix = pd.DataFrame(1/(1+Q), index=cust_prod_pivot.index, columns=cust_prod_pivot.index)
s = similarity_matrix.sort_values(by=[1000], ascending=False).head(6)
list(s.index)

[1000, 17, 4, 21, 29, 49]

In [45]:
total_quant=data.groupby(["CustomerID","ProductName"]).sum()

In [52]:
similar = total_quant.loc[:]
similar

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1
...                                           ...        ...       ...
98200      Vol Au Vents                   5160346        272        50
           Wasabi Powder                  4438427         80        25
           Wine - Fume Blanc Fetzer       6226060         76        25
           Wine - Hardys Bankside Shiraz   266676        360        25
           Yogurt - French Vanilla        2112061        320        25

[63628 rows x 3 columns]

In [53]:
top_products = similar.groupby(["ProductName"]).sum()
top_products.sort_values(by=["Quantity"], ascending = False)

,SalesID,ProductID,Quantity
ProductName,,,
Cookies - Assorted,677818553,14418,2403
Flavouring - Orange,581604928,15660,2397
Fenngreek Seed,629141163,73390,2391
Wine - White Cab Sauv.on,567878568,65360,2387
Bandage - Flexible Neon,617298597,49140,2347
...,...,...,...
Scallops 60/80 Iqf,459611485,14763,1546
Rice - Long Grain,394156698,45593,1531
Kellogs Special K Cereal,420794558,1904,1496


In [55]:
customer_matrix = cust_prod_pivot.T
recomended = customer_matrix.merge(top_products, on="ProductName")
recomended

,0,1,2,3,4,5,6,7,8,9,...,994,995,996,997,998,999,1000,SalesID,ProductID,Quantity
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,482817525,20043,1918
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,611869523,55263,2228
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,0.0,0.0,499189817,48574,1880
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,25.0,0.0,25.0,0.0,0.0,445403740,9928,1738
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,493901835,48600,2035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,430132143,3937,1680
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,571721745,30788,2064
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,528487976,41860,2094


In [57]:
r = recomended[[1000,"Quantity"]]
list(r.loc[r[1000] == 0].sort_values(by=["Quantity"], ascending=False).head(5).index)

['Wine - White, Mosel Gold',
 'Tea - Jasmin Green',
 'Kiwi',
 'Cheese - Brie,danish',
 'Soup Knorr Chili With Beans']

### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [ ]:
product_list=[]
for p in prodcuts:
    similar_products = list(similarity_matrix.sort_values(by=[p], ascending=False).head(6).index)
    similar_products = similar_products[1:-1]
    product_list.append(similar_products)
product_list = [p for x in product_list for p in x]
popular_products = dict(Counter(product_list))
popular_products = sorted(popular_products.items(), key=lambda x: x[1], reverse=True)
popular_products = popular_products[:5]
l=[]
for i in range(len(popular_products)): 
    l.append(popular_products[i][0])
l